<a href="https://colab.research.google.com/github/tacky726/GCI-Competition/blob/main/credit_default.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ライブラリの読み込み
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
from sklearn.preprocessing import MinMaxScaler
warnings.filterwarnings('ignore')

KeyboardInterrupt: 

必要なデータの読み込みを行います。GCIの教材フォルダの構成を想定して、読み込んでいます。

In [ ]:
# Google Colaboratoryで作業する場合はこちらも実行してください。
from google.colab import drive
drive.mount('/content/drive')
# %cd 以降にこのnotebookを置いているディレクトリを指定してください。
%cd "/content/drive/MyDrive"
import pandas as pd
!pip install pycaret


from pycaret.classification import setup, compare_models,  create_model
from pycaret.regression import *

In [ ]:
!pip install category_encoders

In [ ]:

# データの読み込み
# INPUT_DIRにtrain.csvなどのデータを置いているディレクトリを指定してください。
INPUT_DIR = "./input/"
train = pd.read_csv(INPUT_DIR + "train.csv")
test = pd.read_csv(INPUT_DIR + "test.csv")
sample_sub = pd.read_csv(INPUT_DIR + "sample_submission.csv")
# 省略しないように設定
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)
train = train.copy()
test = test.copy()

In [ ]:
# SK_ID_CURR列の10000ごとのグループを作成
train['SK_ID_CURR_group'] = (train['SK_ID_CURR'] // 20000) * 20000

# グループごとのTARGETの平均を計算
target_grouped_mean = train.groupby('SK_ID_CURR_group')['TARGET'].mean()

# 結果を表示
print(target_grouped_mean)

In [ ]:
# データフレームの行数と列数を調べる
num_rows, num_columns = train.shape

# 結果を表示
print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_columns}")

In [ ]:
# データフレームを真ん中で分ける
mid_index_train = len(train) // 2
mid_index_test = len(test) // 2

# 1つ目の部分に0を、2つ目の部分に1を与える列を作成
train['Label'] = [0] * mid_index_train + [1] * (len(train) - mid_index_train)
test['Label'] = [0] * mid_index_test + [1] * (len(test) - mid_index_test)

In [ ]:
import pandas as pd


# SK_ID_CURR列の10000ごとのグループを作成
train['SK_ID_CURR_group'] = (train['SK_ID_CURR'] // 10000) * 10000

# 数値列のみを対象にしてグループごとの平均を計算
numeric_cols = train.select_dtypes(include='number').columns
grouped_mean_all_features = train.groupby('SK_ID_CURR_group')[numeric_cols].mean()

# 各列での最大値と最小値の差を計算
max_min_diff = grouped_mean_all_features.max() - grouped_mean_all_features.min()

# 全データの平均を計算
overall_mean = train[numeric_cols].mean()

# max_min_diffをoverall_meanで割る
ratio = max_min_diff / overall_mean

# ratioが0.1以上の特徴量のみを抜き出す
significant_features = ratio[ratio >= 0.1].index

# significant_featuresに含まれる特徴量の10000ごとの平均を計算
significant_features_grouped_mean = train.groupby('SK_ID_CURR_group')[significant_features].mean()

# 結果を表示
print(significant_features_grouped_mean)

In [ ]:
# CNT_FAM_MEMBERS列からCNT_CHILDREN列を引いた列を追加
train['ADULT_MEMBERS'] = train['CNT_FAM_MEMBERS'] - train['CNT_CHILDREN']
test['ADULT_MEMBERS'] = test['CNT_FAM_MEMBERS'] - test['CNT_CHILDREN']

# 複合グラフ作成
plt.figure(figsize=(10, 6))

# ヒストグラム（TARGET = 0 の場合）
train[train['TARGET'] == 0]['ADULT_MEMBERS'].plot(kind='hist', bins=50, alpha=0.5, color='blue', label='TARGET = 0')

# ヒストグラム（TARGET = 1 の場合）
train[train['TARGET'] == 1]['ADULT_MEMBERS'].plot(kind='hist', bins=50, alpha=0.5, color='red', label='TARGET = 1')

# 散布図
plt.scatter(train['ADULT_MEMBERS'], train['TARGET'], c=train['TARGET'].apply(lambda x: 'red' if x == 1 else 'blue'), alpha=0.5)

plt.xlabel('ADULT_MEMBERS')
plt.ylabel('Count / TARGET')
plt.title('Relationship between ADULT_MEMBERS and TARGET')
plt.legend()
plt.grid(True)
plt.show()
# ADULT_MEMBERSごとのTARGET列の平均を計算
result = train.groupby('ADULT_MEMBERS')['TARGET'].mean()
# 結果の表示
print(result)
# ADULT_MEMBERS列の値を変換する関数
def convert_adult_members(df):
    df['ADULT_MEMBERS'] = df['ADULT_MEMBERS'].apply(lambda x: 1 if x == 2.0 else 0)
    return df

# 変換を適用
train = convert_adult_members(train)
test = convert_adult_members(test)


In [ ]:
# 新しい列を作成する関数
def create_new_flag(df):
    df['NEW_FLAG1'] = df.apply(lambda row: 1 if row['FLAG_WORK_PHONE'] == 0 and (row['FLAG_PHONE'] == 1 or row['FLAG_EMAIL'] == 1) else 0, axis=1)
    return df

# 新しい列を作成
train = create_new_flag(train)
test = create_new_flag(test)

# NEW_FLAGごとのTARGETの平均を計算
train_mean = train.groupby('NEW_FLAG1')['TARGET'].mean().reset_index()
print(train_mean)

In [ ]:
# 指定された列の組み合わせごとにTARGETの平均を計算
result = train.groupby(['LIVE_CITY_NOT_WORK_CITY', 'REG_CITY_NOT_LIVE_CITY'])['TARGET'].mean().reset_index()

# 結果を表示
print(result)

In [ ]:
# 指定された列の組み合わせごとにTARGETの平均を計算
result = train.groupby(['REG_CITY_NOT_WORK_CITY', 'REGION_RATING_CLIENT_W_CITY'])['TARGET'].mean().reset_index()

# 結果を表示
print(result)

In [ ]:
# 新しい列を作成
train['new_column2'] = ((train['LIVE_CITY_NOT_WORK_CITY'] == 0) & (train['REGION_RATING_CLIENT_W_CITY'] == 1)).astype(int)
test['new_column2'] = ((test['LIVE_CITY_NOT_WORK_CITY'] == 0) & (test['REGION_RATING_CLIENT_W_CITY'] == 1)).astype(int)
# 新しい列を作成
train['new_column3'] = ((train['LIVE_CITY_NOT_WORK_CITY'] == 0) & (train['REGION_RATING_CLIENT_W_CITY'] == 2)).astype(int)
test['new_column3'] = ((test['LIVE_CITY_NOT_WORK_CITY'] == 0) & (test['REGION_RATING_CLIENT_W_CITY'] == 2)).astype(int)
# 新しい列を作成
train['new_column4'] = ((train['LIVE_CITY_NOT_WORK_CITY'] == 0) & (train['REGION_RATING_CLIENT_W_CITY'] == 3)).astype(int)
test['new_column4'] = ((test['LIVE_CITY_NOT_WORK_CITY'] == 0) & (test['REGION_RATING_CLIENT_W_CITY'] == 3)).astype(int)
# 新しい列を作成
train['new_column5'] = ((train['LIVE_CITY_NOT_WORK_CITY'] == 1) & (train['REGION_RATING_CLIENT_W_CITY'] == 1)).astype(int)
test['new_column5'] = ((test['LIVE_CITY_NOT_WORK_CITY'] == 1) & (test['REGION_RATING_CLIENT_W_CITY'] == 1)).astype(int)
# 新しい列を作成
train['new_column6'] = ((train['LIVE_CITY_NOT_WORK_CITY'] == 1) & (train['REGION_RATING_CLIENT_W_CITY'] == 2)).astype(int)
test['new_column6'] = ((test['LIVE_CITY_NOT_WORK_CITY'] == 1) & (test['REGION_RATING_CLIENT_W_CITY'] == 2)).astype(int)
# 新しい列を作成
train['new_column7'] = ((train['LIVE_CITY_NOT_WORK_CITY'] == 1) & (train['REGION_RATING_CLIENT_W_CITY'] == 3)).astype(int)
test['new_column7'] = ((test['LIVE_CITY_NOT_WORK_CITY'] == 1) & (test['REGION_RATING_CLIENT_W_CITY'] == 3)).astype(int)
# 新しい列を作成
train['new_column8'] = ((train['REG_CITY_NOT_WORK_CITY'] == 0) & (train['REGION_RATING_CLIENT_W_CITY'] == 1)).astype(int)
test['new_column8'] = ((test['REG_CITY_NOT_WORK_CITY'] == 0) & (test['REGION_RATING_CLIENT_W_CITY'] == 1)).astype(int)
# 新しい列を作成
train['new_column9'] = ((train['REG_CITY_NOT_WORK_CITY'] == 0) & (train['REGION_RATING_CLIENT_W_CITY'] == 2)).astype(int)
test['new_column9'] = ((test['REG_CITY_NOT_WORK_CITY'] == 0) & (test['REGION_RATING_CLIENT_W_CITY'] == 2)).astype(int)
# 新しい列を作成
train['new_column10'] = ((train['REG_CITY_NOT_WORK_CITY'] == 0) & (train['REGION_RATING_CLIENT_W_CITY'] == 3)).astype(int)
test['new_column10'] = ((test['REG_CITY_NOT_WORK_CITY'] == 0) & (test['REGION_RATING_CLIENT_W_CITY'] == 3)).astype(int)
# 新しい列を作成
train['new_column11'] = ((train['REG_CITY_NOT_WORK_CITY'] == 1) & (train['REGION_RATING_CLIENT_W_CITY'] == 1)).astype(int)
test['new_column11'] = ((test['REG_CITY_NOT_WORK_CITY'] == 1) & (test['REGION_RATING_CLIENT_W_CITY'] == 1)).astype(int)
# 新しい列を作成
train['new_column12'] = ((train['REG_CITY_NOT_WORK_CITY'] == 1) & (train['REGION_RATING_CLIENT_W_CITY'] == 2)).astype(int)
test['new_column12'] = ((test['REG_CITY_NOT_WORK_CITY'] == 1) & (test['REGION_RATING_CLIENT_W_CITY'] == 2)).astype(int)
# 新しい列を作成
train['new_column13'] = ((train['REG_CITY_NOT_WORK_CITY'] == 1) & (train['REGION_RATING_CLIENT_W_CITY'] == 3)).astype(int)
test['new_column13'] = ((test['REG_CITY_NOT_WORK_CITY'] == 1) & (test['REGION_RATING_CLIENT_W_CITY'] == 3)).astype(int)

# 新しい列を作成
train['new_column14'] = ((train['REG_CITY_NOT_WORK_CITY'] == 0) & (train['LIVE_CITY_NOT_WORK_CITY'] == 0)).astype(int)
test['new_column14'] = ((test['REG_CITY_NOT_WORK_CITY'] == 0) & (test['LIVE_CITY_NOT_WORK_CITY'] == 0)).astype(int)
# 新しい列を作成
train['new_column15'] = ((train['REG_CITY_NOT_WORK_CITY'] == 0) & (train['LIVE_CITY_NOT_WORK_CITY'] == 1)).astype(int)
test['new_column15'] = ((test['REG_CITY_NOT_WORK_CITY'] == 0) & (test['LIVE_CITY_NOT_WORK_CITY'] == 1)).astype(int)
# 新しい列を作成
train['new_column16'] = ((train['REG_CITY_NOT_WORK_CITY'] == 1) & (train['LIVE_CITY_NOT_WORK_CITY'] == 0)).astype(int)
test['new_column16'] = ((test['REG_CITY_NOT_WORK_CITY'] == 1) & (test['LIVE_CITY_NOT_WORK_CITY'] == 0)).astype(int)
# 新しい列を作成
train['new_column17'] = ((train['REG_CITY_NOT_WORK_CITY'] == 1) & (train['LIVE_CITY_NOT_WORK_CITY'] == 1)).astype(int)
test['new_column17'] = ((test['REG_CITY_NOT_WORK_CITY'] == 1) & (test['LIVE_CITY_NOT_WORK_CITY'] == 1)).astype(int)

## 1. データの可視化と分析

In [ ]:
# trainデータの確認
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
print(f"train shape: {train.shape}")
# SK_ID_CURR列の10000ごとのグループを作成
train['SK_ID_CURR_group'] = (train['SK_ID_CURR'] // 10000) * 10000

# 数値列のみを対象にしてグループごとの平均を計算
numeric_cols = train.select_dtypes(include='number').columns
grouped_mean_all_features = train.groupby('SK_ID_CURR_group')[numeric_cols].mean()
# 結果を表示
print(grouped_mean_all_features)

In [ ]:
# DAYS_LAST_PHONE_CHANGE列のNaNを0に置き換え、絶対値を計算
train['ABS_DAYS_LAST_PHONE_CHANGE'] = train['DAYS_LAST_PHONE_CHANGE'].abs().fillna(99999)

# AMT_REQ_CREDIT_BUREAU_MON列に30をかけた値を計算
train['AMT_REQ_CREDIT_BUREAU_MON_30'] = train['AMT_REQ_CREDIT_BUREAU_MON'] * 30

# 絶対値+-7の範囲内にAMT_REQ_CREDIT_BUREAU_MON_30の値が入っているかどうかを判定するブール列を作成
train['IN_RANGE'] = np.abs(train['ABS_DAYS_LAST_PHONE_CHANGE'] - train['AMT_REQ_CREDIT_BUREAU_MON_30']) <= 0

# IN_RANGEがTrueの行のTARGETの平均を計算
target_mean = train.loc[train['IN_RANGE'], 'TARGET'].mean()

print(f'TARGETの平均: {target_mean}')

# IN_RANGE列のTrueの数を数える
in_range_count = train['IN_RANGE'].sum()

print(f'IN_RANGE列のTrueの数: {in_range_count}')

In [ ]:
# trainデータフレームのTARGETの全平均を計算
target_mean_all_actual = train['TARGET'].mean()

print(f'TARGETの全平均: {target_mean_all_actual}')

In [ ]:
def create_match_column(df):
    # DAYS_LAST_PHONE_CHANGE列のNaNを0に置き換え、絶対値を計算
    df['ABS_DAYS_LAST_PHONE_CHANGE'] = df['DAYS_LAST_PHONE_CHANGE'].fillna(0).abs()

    # AMT_REQ_CREDIT_BUREAU_MON列に30をかけた値を計算
    df['AMT_REQ_CREDIT_BUREAU_MON_30'] = df['AMT_REQ_CREDIT_BUREAU_MON'] * 30

    # 絶対値が一致するかどうかを判定するブール列を作成し、整数に変換
    df['MATCH'] = (df['ABS_DAYS_LAST_PHONE_CHANGE'] == df['AMT_REQ_CREDIT_BUREAU_MON_30']).astype(int)
    return df

# trainとtestデータフレームに対して新しい列を作成
train = create_match_column(train)
test = create_match_column(test)

In [ ]:
# testデータの確認
print(f"test shape: {test.shape}")
test.head(3)

In [ ]:
# 指定された列を削除
train.drop(columns=['ABS_DAYS_LAST_PHONE_CHANGE', 'AMT_REQ_CREDIT_BUREAU_MON_30'], inplace=True)
test.drop(columns=['ABS_DAYS_LAST_PHONE_CHANGE', 'AMT_REQ_CREDIT_BUREAU_MON_30'], inplace=True)

In [ ]:
# 数値データのみを抽出する
numeric_data = train.select_dtypes(include=['number'])
# 相関行列を計算
correlation_matrix = numeric_data.corr()

# 相関行列を出力
print("Correlation Matrix:")
print(correlation_matrix)

In [ ]:
# AMT_INCOME_TOTALを1000単位のビンに分ける
from matplotlib.ticker import MaxNLocator
# 外れ値を除外する。ここでは、1パーセンタイルと99パーセンタイルを基準に外れ値を除外します。
q_low = train['AMT_INCOME_TOTAL'].quantile(0.01)
q_high = train['AMT_INCOME_TOTAL'].quantile(0.99)
train_filtered = train[(train['AMT_INCOME_TOTAL'] >= q_low) & (train['AMT_INCOME_TOTAL'] <= q_high)]

# AMT_INCOME_TOTALを1000単位のビンに分ける
bins = range(0, int(train_filtered['AMT_INCOME_TOTAL'].max()) + 10000, 10000)
labels = [f'{i}-{i+10000}' for i in bins[:-1]]
train_filtered['INCOME_BIN'] = pd.cut(train_filtered['AMT_INCOME_TOTAL'], bins=bins, labels=labels, right=False)

# INCOME_BINごとにTARGETの平均を計算
income_target = train_filtered.groupby('INCOME_BIN')['TARGET'].mean()

# 棒グラフを作成
fig, ax = plt.subplots(figsize=(12, 6))
income_target.plot(kind='bar', ax=ax)
plt.xlabel('Income Range')
plt.ylabel('Average TARGET')
plt.title('Average TARGET by Income Range')
ax.xaxis.set_major_locator(MaxNLocator(nbins=15))  # x軸の目盛りを間引く
plt.xticks(rotation=45, fontsize=10)  # フォントサイズを指定
plt.show()


In [ ]:
count = train[train['AMT_INCOME_TOTAL'] > 2000000].shape[0]

# 結果を表示
print("AMT_INCOME_TOTAL列が0.2を超えている人数:", count)

In [ ]:
# trainの欠損値を確認
train.isnull().sum()

In [ ]:
# 新しい列 'EXT_SOURCE_1_IS_NAN' を作成
train['EXT_SOURCE_1_IS_NAN'] = train['EXT_SOURCE_1'].isna().astype(int)
test['EXT_SOURCE_1_IS_NAN'] = test['EXT_SOURCE_1'].isna().astype(int)
# 新しい列 'EXT_SOURCE_3_IS_NAN' を作成
train['EXT_SOURCE_3_IS_NAN'] = train['EXT_SOURCE_3'].isna().astype(int)
test['EXT_SOURCE_3_IS_NAN'] = test['EXT_SOURCE_3'].isna().astype(int)
# 新しい列 'EXT_SOURCE_3_IS_NAN' を作成
train['OCCUPATION_TYPE_NAN'] = train['OCCUPATION_TYPE'].isna().astype(int)
test['OCCUPATION_TYPE_NAN'] = test['OCCUPATION_TYPE'].isna().astype(int)
# 新しい列 'EXT_SOURCE_3_IS_NAN' を作成
train['OWN_CAR_AGE_NAN'] = train['OWN_CAR_AGE'].isna().astype(int)
test['OWN_CAR_AGE_NAN'] = test['OWN_CAR_AGE'].isna().astype(int)
# 新しい列 'EXT_SOURCE_3_IS_NAN' を作成
train['AMT_REQ_CREDIT_BUREAU_HOUR_NAN'] = train['AMT_REQ_CREDIT_BUREAU_HOUR'].isna().astype(int)
test['AMT_REQ_CREDIT_BUREAU_HOUR_NAN'] = test['AMT_REQ_CREDIT_BUREAU_HOUR'].isna().astype(int)

In [ ]:
# testの欠損値を確認
test.isnull().sum()

In [ ]:
# EXT_SOURCE_1 列が欠損値の行を抽出
train_missing_ext_source_1 = train[train['FLAG_OWN_REALTY'].isna()]

# EXT_SOURCE_1 列が埋めてある行を抽出
train_filled_ext_source_1 = train[~train['FLAG_OWN_REALTY'].isna()]
import matplotlib.pyplot as plt
import seaborn as sns

# TARGET 列の分布を比較
plt.figure(figsize=(12, 6))

# EXT_SOURCE_1 が欠損値の行の TARGET 分布
plt.subplot(1, 2, 1)
sns.countplot(data=train_missing_ext_source_1, x='TARGET')
plt.title('TARGET Distribution (FLAG_OWN_REALTY  is NaN)')
plt.xlabel('TARGET')
plt.ylabel('Count')

# EXT_SOURCE_1 が埋めてある行の TARGET 分布
plt.subplot(1, 2, 2)
sns.countplot(data=train_filled_ext_source_1, x='TARGET')
plt.title('TARGET Distribution (FLAG_OWN_REALTY is Filled)')
plt.xlabel('TARGET')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
test.shape

In [ ]:
# EXT_SOURCE_1 列が欠損値の行を抽出
train_missing_ext_source_1 = train[train['OCCUPATION_TYPE'].isna()]

# EXT_SOURCE_1 列が埋めてある行を抽出
train_filled_ext_source_1 = train[~train['OCCUPATION_TYPE'].isna()]
import matplotlib.pyplot as plt
import seaborn as sns

# TARGET 列の分布を比較
plt.figure(figsize=(12, 6))

# EXT_SOURCE_1 が欠損値の行の TARGET 分布
plt.subplot(1, 2, 1)
sns.countplot(data=train_missing_ext_source_1, x='TARGET')
plt.title('TARGET Distribution (OCCUPATION_TYPE is NaN)')
plt.xlabel('TARGET')
plt.ylabel('Count')

# EXT_SOURCE_1 が埋めてある行の TARGET 分布
plt.subplot(1, 2, 2)
sns.countplot(data=train_filled_ext_source_1, x='TARGET')
plt.title('TARGET Distribution (OCCUPATION_TYPE is Filled)')
plt.xlabel('TARGET')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
# 数値データ以外の列を削除する
numeric_columns = train.select_dtypes(include=['number']).columns
numeric_data = train[numeric_columns]

# 相関行列を計算
correlation_matrix = numeric_data.corr()

# 相関行列を出力
print("Correlation Matrix:")
print(correlation_matrix)

In [ ]:
# EXT_SOURCE_1 列が欠損値の行を抽出
train_missing_ext_source_1 = train[train['EXT_SOURCE_1'].isna()]

# EXT_SOURCE_1 列が埋めてある行を抽出
train_filled_ext_source_1 = train[~train['EXT_SOURCE_1'].isna()]
import matplotlib.pyplot as plt
import seaborn as sns

# TARGET 列の分布を比較
plt.figure(figsize=(12, 6))

# EXT_SOURCE_1 が欠損値の行の TARGET 分布
plt.subplot(1, 2, 1)
sns.countplot(data=train_missing_ext_source_1, x='TARGET')
plt.title('TARGET Distribution (EXT_SOURCE_1 is NaN)')
plt.xlabel('TARGET')
plt.ylabel('Count')

# EXT_SOURCE_1 が埋めてある行の TARGET 分布
plt.subplot(1, 2, 2)
sns.countplot(data=train_filled_ext_source_1, x='TARGET')
plt.title('TARGET Distribution (EXT_SOURCE_1 is Filled)')
plt.xlabel('TARGET')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
# EXT_SOURCE_1 列が欠損値の行を抽出
train_missing_ext_source_1 = train[train['AMT_REQ_CREDIT_BUREAU_HOUR'].isna()]

# EXT_SOURCE_1 列が埋めてある行を抽出
train_filled_ext_source_1 = train[~train['AMT_REQ_CREDIT_BUREAU_HOUR'].isna()]
import matplotlib.pyplot as plt
import seaborn as sns

# TARGET 列の分布を比較
plt.figure(figsize=(12, 6))

# EXT_SOURCE_1 が欠損値の行の TARGET 分布
plt.subplot(1, 2, 1)
sns.countplot(data=train_missing_ext_source_1, x='TARGET')
plt.title('TARGET Distribution (AMT_REQ_CREDIT_BUREAU_HOUR is NaN)')
plt.xlabel('TARGET')
plt.ylabel('Count')

# EXT_SOURCE_1 が埋めてある行の TARGET 分布
plt.subplot(1, 2, 2)
sns.countplot(data=train_filled_ext_source_1, x='TARGET')
plt.title('TARGET Distribution (AMT_REQ_CREDIT_BUREAU_HOUR is Filled)')
plt.xlabel('TARGET')
plt.ylabel('Count')

plt.tight_layout()
plt.show()

In [ ]:
columns_to_remove = [ 4, 5, 6, 7, 22,  34, 42, 44, 47,48,49,50]

# Remove the specified columns
train.drop(train.columns[columns_to_remove], axis=1, inplace=True)

# Specify the columns to be removed, adjusted by -1 (so, 1~6, 21, 24, 28, 33, 41, 43, 46)
columns_to_remove = [3, 4, 5, 6, 21, 33, 41, 43, 46,47,48,49]



# Remove the specified columns from test
test.drop(test.columns[columns_to_remove], axis=1, inplace=True)

In [ ]:
# AMT_CREDITをAMT_GOODS_PRICEで割った列を追加
train['CREDIT_GOODS_RATIO'] = train['AMT_CREDIT'] / train['AMT_GOODS_PRICE']
test['CREDIT_GOODS_RATIO'] = test['AMT_CREDIT'] / test['AMT_GOODS_PRICE']
# グラフ作成
plt.figure(figsize=(10, 6))
colors = {0: 'blue', 1: 'red'}
plt.scatter(train['CREDIT_GOODS_RATIO'], train['TARGET'], c=train['TARGET'].apply(lambda x: colors[x]), alpha=0.5)
plt.xlabel('CREDIT_GOODS_RATIO')
plt.ylabel('TARGET')
plt.title('Relationship between CREDIT_GOODS_RATIO and TARGET')
plt.grid(True)
plt.show()

# CREDIT_GOODS_RATIO列に条件に基づいて値を設定
train['CREDIT_GOODS_RATIO'] = train['CREDIT_GOODS_RATIO'].apply(lambda x: 0 if x <= 1 else 1)
test['CREDIT_GOODS_RATIO'] = test['CREDIT_GOODS_RATIO'].apply(lambda x: 0 if x <= 1 else 1)

# AMT_CREDITをAMT_ANNUITYで割った列を追加
train['CREDIT_ANNUITY_RATIO'] = train['AMT_CREDIT'] / train['AMT_ANNUITY']
test['CREDIT_ANNUITY_RATIO'] = test['AMT_CREDIT'] / test['AMT_ANNUITY']


# 複合グラフ作成
plt.figure(figsize=(10, 6))

# ヒストグラム（TARGET = 0 の場合）
train[train['TARGET'] == 0]['CREDIT_ANNUITY_RATIO'].plot(kind='hist', bins=50, alpha=0.5, color='blue', label='TARGET = 0')

# ヒストグラム（TARGET = 1 の場合）
train[train['TARGET'] == 1]['CREDIT_ANNUITY_RATIO'].plot(kind='hist', bins=50, alpha=0.5, color='red', label='TARGET = 1')

# 散布図
plt.scatter(train['CREDIT_ANNUITY_RATIO'], train['TARGET'], c=train['TARGET'].apply(lambda x: 'red' if x == 1 else 'blue'), alpha=0.5)

plt.xlabel('CREDIT_ANNUITY_RATIO')
plt.ylabel('Count / TARGET')
plt.title('Relationship between CREDIT_ANNUITY_RATIO and TARGET')
plt.legend()
plt.grid(True)
plt.show()

# NaN の数を数える
nan_count_train = train['CREDIT_ANNUITY_RATIO'].isna().sum()
nan_count_test = test['CREDIT_ANNUITY_RATIO'].isna().sum()

# 無限大の数を数える
inf_count_train = np.isinf(train['CREDIT_ANNUITY_RATIO']).sum()
inf_count_test = np.isinf(test['CREDIT_ANNUITY_RATIO']).sum()

# 結果を表示
print(f"Train DataFrame - NaN count: {nan_count_train}, Inf count: {inf_count_train}")
print(f"Test DataFrame - NaN count: {nan_count_test}, Inf count: {inf_count_test}")

# 無限大の値を NaN に置き換える
train['CREDIT_ANNUITY_RATIO'].replace([np.inf, -np.inf], np.nan, inplace=True)
test['CREDIT_ANNUITY_RATIO'].replace([np.inf, -np.inf], np.nan, inplace=True)

# NaN の値を 0 に置き換える
train['CREDIT_ANNUITY_RATIO'].fillna(0, inplace=True)
test['CREDIT_ANNUITY_RATIO'].fillna(0, inplace=True)
# CREDIT_ANNUITY_RATIOの整数部分を計算して新しい列に追加
train['CREDIT_ANNUITY_RATIO_INT'] = train['CREDIT_ANNUITY_RATIO'].astype(int)

# CREDIT_ANNUITY_RATIOの整数値ごとのTARGETの平均を計算
ratio_target_mean = train.groupby('CREDIT_ANNUITY_RATIO_INT')['TARGET'].mean()
# 結果を表示
print(ratio_target_mean)
# 列の削除
train.drop(columns=['CREDIT_ANNUITY_RATIO_INT'], inplace=True)

# CREDIT_ANNUITY_RATIO_INT列が11以上21以下のときに1、そうでないときに0を設定
train['CREDIT_ANNUITY_RATIO'] = train['CREDIT_ANNUITY_RATIO'].apply(lambda x: 1 if 11 <= x <= 21 else 0)
test['CREDIT_ANNUITY_RATIO'] = test['CREDIT_ANNUITY_RATIO'].apply(lambda x: 1 if 11 <= x <= 21 else 0)


In [ ]:

import category_encoders as ce

list_cols = ['NAME_INCOME_TYPE','NAME_FAMILY_STATUS','REGION_RATING_CLIENT','REGION_RATING_CLIENT_W_CITY','CODE_GENDER','NAME_CONTRACT_TYPE','DEF_30_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE',]

# OneHotEncodeしたい列を指定。Nullや不明の場合の補完方法も指定。
ce_ohe = ce.OneHotEncoder(cols=list_cols,handle_unknown='impute')

# pd.DataFrameをそのまま突っ込む
train = ce_ohe.fit_transform(train)
test = ce_ohe.fit_transform(test)

# 対象の住宅タイプのリスト
target_housing_types = ['With parents', 'Rented apartment']

# 新しい列の作成
train['HOUSING_FLAG'] = train['NAME_HOUSING_TYPE'].apply(lambda x: 1 if x in target_housing_types else 0)
test['HOUSING_FLAG'] = test['NAME_HOUSING_TYPE'].apply(lambda x: 1 if x in target_housing_types else 0)

# trainデータでNAME_EDUCATION_TYPEごとのTARGETの平均を計算
education_target_mean = train.groupby('NAME_EDUCATION_TYPE')['TARGET'].mean().reset_index()
education_target_mean.columns = ['NAME_EDUCATION_TYPE', 'EDUCATION_TARGET_MEAN']

# trainデータに新しい列を追加
train = pd.merge(train, education_target_mean, on='NAME_EDUCATION_TYPE', how='left')

# testデータに新しい列を追加
test = pd.merge(test, education_target_mean, on='NAME_EDUCATION_TYPE', how='left')

# trainデータでORGANIZATION_TYPEごとのTARGETの平均を計算
organization_target_mean = train.groupby('ORGANIZATION_TYPE')['TARGET'].mean().reset_index()
organization_target_mean.columns = ['ORGANIZATION_TYPE', 'ORGANIZATION_TARGET_MEAN']

# trainデータに新しい列を追加
train = pd.merge(train, organization_target_mean, on='ORGANIZATION_TYPE', how='left')

# testデータに新しい列を追加
test = pd.merge(test, organization_target_mean, on='ORGANIZATION_TYPE', how='left')

# trainデータでOCCUPATION_TYPEごとのTARGETの平均を計算
occupation_target_mean = train.groupby('OCCUPATION_TYPE')['TARGET'].mean().reset_index()
occupation_target_mean.columns = ['OCCUPATION_TYPE', 'OCCUPATION_TARGET_MEAN']

# trainデータに新しい列を追加
train = pd.merge(train, occupation_target_mean, on='OCCUPATION_TYPE', how='left')

# testデータに新しい列を追加
test = pd.merge(test, occupation_target_mean, on='OCCUPATION_TYPE', how='left')
# 削除したい列名を指定
columns_to_drop = ['ORGANIZATION_TYPE','OCCUPATION_TYPE','NAME_HOUSING_TYPE','NAME_EDUCATION_TYPE','NAME_TYPE_SUITE']
# 指定の列を削除
train = train.drop(columns=columns_to_drop)
test = test.drop(columns=columns_to_drop)
# TARGET列を除いたtrainデータの特徴量部分を抽出
train_features = train.drop(columns=['TARGET'])

# trainの列名とtestの列名を取得
train_columns = train_features.columns
test_columns = test.columns

# trainにあってtestにない列を見つける
missing_columns = set(train_columns) - set(test_columns)

# testデータに欠けている列を追加し、すべての値を0に設定
for column in missing_columns:
    test[column] = 0

# 列の順序をtrainデータと同じにする
test = test[train_columns]

# 0と1の比率が90%以上の列を見つけるための閾値
threshold = 0.95

# 偏りのある列を探す関数
def find_biased_columns(df, threshold):
    biased_columns = []
    for column in df.columns:
        counts = df[column].value_counts(normalize=True)
        if (0 in counts and counts[0] > threshold) or (1 in counts and counts[1] > threshold):
            biased_columns.append(column)
    return biased_columns

# trainとtestの両方で偏りのある列を探す
biased_columns_train = find_biased_columns(train, threshold)
biased_columns_test = find_biased_columns(test, threshold)
biased_columns = list(set(biased_columns_train) | set(biased_columns_test))

# 偏りのある列を削除
train = train.drop(columns=biased_columns)
test = test.drop(columns=biased_columns)


In [ ]:
train.head()

In [ ]:
# ブール型データを数値型に変換
train = train.applymap(lambda x: int(x) if isinstance(x, bool) else x)
test = test.applymap(lambda x: int(x) if isinstance(x, bool) else x)
# EXT_SOURCE_1 列が欠損値の行を抽出
train_missing_ext_source_1 = train[train['EXT_SOURCE_3'].isna()]

# EXT_SOURCE_1 列が埋めてある行を抽出
train_filled_ext_source_1 = train[~train['EXT_SOURCE_3'].isna()]
# EXT_SOURCE_1 列が欠損値の行を抽出
test_missing_ext_source_1 = test[test['EXT_SOURCE_3'].isna()]

# EXT_SOURCE_1 列が埋めてある行を抽出
test_filled_ext_source_1 = test[~test['EXT_SOURCE_3'].isna()]
# 埋めてある行を結合
filled_ext_source_1 = pd.concat([train_filled_ext_source_1, test_filled_ext_source_1], ignore_index=True)
# 残したい列のリスト
columns_to_keep = ['DAYS_BIRTH', 'EXT_SOURCE_3','EXT_SOURCE_1']




exp1 = setup(data = filled_ext_source_1, target = 'EXT_SOURCE_3', session_id=123,ignore_features = 'TARGET',  use_gpu=True, pca_components=0.95,feature_selection=True, feature_selection_method='univariate') #前処理を実行

model = create_model('lightgbm')
#パラメータチューニング
final_model = finalize_model(model) #モデルの確定
train_missing_ext_source_1 = train_missing_ext_source_1.drop(['EXT_SOURCE_3'], axis=1)
result1 = predict_model(final_model, train_missing_ext_source_1)
# 予測された値を元のデータフレームに埋め込む
train.loc[train['EXT_SOURCE_3'].isna(), 'EXT_SOURCE_3'] = result1['prediction_label']
test_missing_ext_source_1 = test_missing_ext_source_1.drop(['EXT_SOURCE_3'], axis=1)
result2 = predict_model(final_model, test_missing_ext_source_1)
test.loc[test['EXT_SOURCE_3'].isna(), 'EXT_SOURCE_3'] = result2['prediction_label']

In [ ]:
# EXT_SOURCE_1 列が欠損値の行を抽出
train_missing_ext_source_1 = train[train['EXT_SOURCE_1'].isna()]

# EXT_SOURCE_1 列が埋めてある行を抽出
train_filled_ext_source_1 = train[~train['EXT_SOURCE_1'].isna()]

In [ ]:
# EXT_SOURCE_1 列が欠損値の行を抽出
test_missing_ext_source_1 = test[test['EXT_SOURCE_1'].isna()]

# EXT_SOURCE_1 列が埋めてある行を抽出
test_filled_ext_source_1 = test[~test['EXT_SOURCE_1'].isna()]

In [ ]:
# 埋めてある行を結合
filled_ext_source_1 = pd.concat([train_filled_ext_source_1, test_filled_ext_source_1], ignore_index=True)

In [ ]:
# 残したい列のリスト
columns_to_keep = ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'EXT_SOURCE_2', 'EXT_SOURCE_3','EXT_SOURCE_1','FLAG_OWN_CAR','CODE_GENDER',]



In [ ]:
import pandas as pd
from pycaret.classification import setup, compare_models, blend_models, tune_model, evaluate_model, finalize_model
from pycaret.regression import *


In [ ]:
filled_ext_source_1.head()

In [ ]:
exp1 = setup(data = filled_ext_source_1, target = 'EXT_SOURCE_1', session_id=45,ignore_features = 'TARGET',  use_gpu=True, pca_components=0.95,feature_selection=True, feature_selection_method='univariate') #前処理を実行

In [ ]:
model = create_model('lightgbm')

In [ ]:
final_model = finalize_model(model) #モデルの確定

In [ ]:
train_missing_ext_source_1

In [ ]:
train_missing_ext_source_1 = train_missing_ext_source_1.drop(['EXT_SOURCE_1'], axis=1)
result1 = predict_model(final_model, train_missing_ext_source_1)

In [ ]:
# 予測された値を元のデータフレームに埋め込む
train.loc[train['EXT_SOURCE_1'].isna(), 'EXT_SOURCE_1'] = result1['prediction_label']


In [ ]:
test_missing_ext_source_1 = test_missing_ext_source_1.drop(['EXT_SOURCE_1'], axis=1)
result2 = predict_model(final_model, test_missing_ext_source_1)

In [ ]:
test.loc[test['EXT_SOURCE_1'].isna(), 'EXT_SOURCE_1'] = result2['prediction_label']

In [ ]:
test.shape

In [ ]:
test.shape

In [ ]:
test.shape

In [ ]:

# 各列の中央値を計算
median_ext_source_1_train = train['EXT_SOURCE_1'].median()
median_ext_source_2_train = train['EXT_SOURCE_2'].median()
median_ext_source_3_train = train['EXT_SOURCE_3'].median()

median_ext_source_1_test = test['EXT_SOURCE_1'].median()
median_ext_source_2_test = test['EXT_SOURCE_2'].median()
median_ext_source_3_test = test['EXT_SOURCE_3'].median()

# 条件を設定し、新しい列 'EXT_SOURCE_FLAG' を作成
train['EXT_SOURCE_FLAG1'] = ((train['EXT_SOURCE_1'] <= median_ext_source_1_train) &
                            (train['EXT_SOURCE_2'] <= median_ext_source_2_train) &
                            (train['EXT_SOURCE_3'] <= median_ext_source_3_train)).astype(int)

test['EXT_SOURCE_FLAG1'] = ((test['EXT_SOURCE_1'] <= median_ext_source_1_test) &
                           (test['EXT_SOURCE_2'] <= median_ext_source_2_test) &
                           (test['EXT_SOURCE_3'] <= median_ext_source_3_test)).astype(int)

# 各列のパーセンタイル値を計算
quantile_ext_source_1_train = train['EXT_SOURCE_1'].quantile(0.2)
quantile_ext_source_2_train = train['EXT_SOURCE_2'].quantile(1/3)
quantile_ext_source_3_train = train['EXT_SOURCE_3'].quantile(0.2)

quantile_ext_source_1_test = test['EXT_SOURCE_1'].quantile(0.2)
quantile_ext_source_2_test = test['EXT_SOURCE_2'].quantile(1/3)
quantile_ext_source_3_test = test['EXT_SOURCE_3'].quantile(0.2)

# 条件を設定し、新しい列 'EXT_SOURCE_FLAG' を作成
train['EXT_SOURCE_FLAG2'] = ((train['EXT_SOURCE_1'] <= quantile_ext_source_1_train) &
                            (train['EXT_SOURCE_2'] <= quantile_ext_source_2_train) &
                            (train['EXT_SOURCE_3'] <= quantile_ext_source_3_train)).astype(int)

test['EXT_SOURCE_FLAG2'] = ((test['EXT_SOURCE_1'] <= quantile_ext_source_1_test) &
                           (test['EXT_SOURCE_2'] <= quantile_ext_source_2_test) &
                           (test['EXT_SOURCE_3'] <= quantile_ext_source_3_test)).astype(int)


# 各列の3分位点を計算
quantile_ext_source_1_train = train['EXT_SOURCE_1'].quantile(1/3)
quantile_ext_source_2_train = train['EXT_SOURCE_2'].quantile(1/3)
quantile_ext_source_3_train = train['EXT_SOURCE_3'].quantile(1/3)

quantile_ext_source_1_test = test['EXT_SOURCE_1'].quantile(1/3)
quantile_ext_source_2_test = test['EXT_SOURCE_2'].quantile(1/3)
quantile_ext_source_3_test = test['EXT_SOURCE_3'].quantile(1/3)

# 条件を設定し、新しい列 'EXT_SOURCE_FLAG' を作成
train['EXT_SOURCE_FLA3'] = ((train['EXT_SOURCE_1'] <= quantile_ext_source_1_train) &
                            (train['EXT_SOURCE_2'] <= quantile_ext_source_2_train) &
                            (train['EXT_SOURCE_3'] <= quantile_ext_source_3_train)).astype(int)

test['EXT_SOURCE_FLAG3'] = ((test['EXT_SOURCE_1'] <= quantile_ext_source_1_test) &
                           (test['EXT_SOURCE_2'] <= quantile_ext_source_2_test) &
                           (test['EXT_SOURCE_3'] <= quantile_ext_source_3_test)).astype(int)

# 各列の10分位点を計算
quantile_ext_source_1_train = train['EXT_SOURCE_1'].quantile(0.1)
quantile_ext_source_2_train = train['EXT_SOURCE_2'].quantile(0.1)
quantile_ext_source_3_train = train['EXT_SOURCE_3'].quantile(0.1)

quantile_ext_source_1_test = test['EXT_SOURCE_1'].quantile(0.1)
quantile_ext_source_2_test = test['EXT_SOURCE_2'].quantile(0.1)
quantile_ext_source_3_test = test['EXT_SOURCE_3'].quantile(0.1)

# 条件を設定し、新しい列 'EXT_SOURCE_FLAG' を作成
train['EXT_SOURCE_FLAG4'] = ((train['EXT_SOURCE_1'] <= quantile_ext_source_1_train) &
                            (train['EXT_SOURCE_2'] <= quantile_ext_source_2_train) &
                            (train['EXT_SOURCE_3'] <= quantile_ext_source_3_train)).astype(int)

test['EXT_SOURCE_FLAG4'] = ((test['EXT_SOURCE_1'] <= quantile_ext_source_1_test) &
                           (test['EXT_SOURCE_2'] <= quantile_ext_source_2_test) &
                           (test['EXT_SOURCE_3'] <= quantile_ext_source_3_test)).astype(int)


# Eoncodeしたい列をリストで指定。もちろん複数指定可能。
list_cols = ['NAME_INCOME_TYPE','NAME_FAMILY_STATUS','REGION_RATING_CLIENT','REGION_RATING_CLIENT_W_CITY','CODE_GENDER','NAME_CONTRACT_TYPE','DEF_30_CNT_SOCIAL_CIRCLE','DEF_60_CNT_SOCIAL_CIRCLE',]



In [ ]:
train.head()


In [ ]:
# trainデータの確認
# 全体を表示するための設定変更

print(f"train shape: {train.shape}")
train.head(3)

In [ ]:
# testデータの確認
print(f"test shape: {test.shape}")
test.head(3)

In [ ]:
# TARGET列を除いたtrainデータの特徴量部分を抽出
train_features = train.drop(columns=['TARGET'])

# 列名の確認
train_columns = train_features.columns
test_columns = test.columns

print("Train Data Columns (without TARGET):")
print(train_columns)
print("\nTest Data Columns:")
print(test_columns)

# 一致しているかどうかを確認
if set(train_columns) == set(test_columns):
    print("\nThe column names match.")
else:
    print("\nThe column names do not match.")

In [ ]:
print(test.columns)
print(train.columns)

In [ ]:
import pandas as pd
from pycaret.classification import setup, compare_models,  create_model
from pycaret.regression import *

In [ ]:
train = train.drop(columns=['SK_ID_CURR'])
test = test.drop(columns=['SK_ID_CURR'])

In [ ]:
exp1 = setup(data=train,imputation_type= 'iterative', target='TARGET', session_id=123, ignore_features=[], use_gpu=True,pca_components=0.95)


In [ ]:
final_model = finalize_model(model) #モデルの確定

In [ ]:
# 列名の変更
test.rename(columns={'EXT_SOURCE_FLAG3': 'EXT_SOURCE_FLA3'}, inplace=True)

In [ ]:
result1 = predict_model(final_model, test)

In [ ]:
result1


In [ ]:
# 予測結果を提出用のフォーマットに格納
sample_sub['TARGET'] = last_column
# TARGET列のマイナス値を0に変換
sample_sub['TARGET'] = sample_sub['TARGET'].apply(lambda x: 0 if x < 0 else x)
sample_sub.shape

In [ ]:
# 1を超える値があるかどうかを確認
has_values_greater_than_one = (sample_sub['TARGET'] > 1).any()

# 結果の表示
if has_values_greater_than_one:
    print("TARGET列には1を超える値が含まれています。")
else:
    print("TARGET列には1を超える値は含まれていません。")

In [ ]:
# 0以下の値の数を確認
num_values_less_or_equal_zero = (sample_sub['TARGET'] <= 0).sum()

# 結果の表示
print(f"TARGET列には0以下の値が {num_values_less_or_equal_zero} 個含まれています。")


In [ ]:
# 提出用のcsvファイルを作成
sample_sub.to_csv('submission.csv',index=False)

以上で、Home Credit Default Riskコンペのチュートリアルは終了です。今回は、50種類ある特徴量のうち5種類しか使用していないので、まだまだ改善の余地があります。この後は、このnotebookやこれまでの教材を参考にして、さらなるスコアの向上を目指してください！